In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [24]:
import os
import numpy as np
import shutil
import cv2
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

In [6]:
!unzip -n /content/drive/MyDrive/dataset/Run.zip -d /content/drive/MyDrive/dataset/Run/

Archive:  /content/drive/MyDrive/dataset/Run.zip
  inflating: /content/drive/MyDrive/dataset/Run/Image_10017.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10032.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10038.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10050.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10054.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10058.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10060.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10100.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10108.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10116.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10117.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_1012.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10126.jpg  
  inflating: /content/drive/MyDrive/dataset/Run/Image_10183.jpg  
  inflating: /content/drive/

In [7]:
!unzip -n /content/drive/MyDrive/dataset/Walk.zip -d /content/drive/MyDrive/dataset/Walk/

Archive:  /content/drive/MyDrive/dataset/Walk.zip
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV1_jpg.rf.44320adf51a34f6f6fe3f94b8285c9b0.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV1_jpg.rf.a0fee6245c425ebbfb3ee4435baf5eaf.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV10_jpg.rf.299dea720f0808676e2f63f6134c29bd.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV10_jpg.rf.d5adee0f0780b89d924b760f81335fd3.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV100_jpg.rf.0341ee76449d74b7ddbda2e73ce1bc2b.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV100_jpg.rf.c125896fd3b3bc7087ad75f07e635937.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV101_jpg.rf.22723134672112473470fc4f3cd87bca.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV101_jpg.rf.f9c9638a77d107c66f7a98f5562ccc5d.jpg  
  inflating: /content/drive/MyDrive/dataset/Walk/humanwalkV102_jpg.rf.0486df5757dd96

In [8]:
data_dir = '/content/drive/MyDrive/dataset'
train_dir = '/content/drive/MyDrive/dataset/train'
test_dir = '/content/drive/MyDrive/dataset/test'
categories = ['Run', 'Walk']

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

for category in categories:
    category_path = os.path.join(data_dir, category)
    images = os.listdir(category_path)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=1)

    for img in train_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(train_dir, category, img))

    for img in test_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(test_dir, category, img))

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 2542 images belonging to 2 classes.
Found 634 images belonging to 2 classes.
Found 795 images belonging to 2 classes.


In [20]:
def create_model(hp):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    base_model.trainable = False

    model = Sequential()
    model.add(base_model)

    for i in range(hp.Int('conv_blocks', 1, 3, default=2)):
        model.add(Conv2D(
            filters=hp.Int(f'conv_{i}_filters', min_value=32, max_value=256, step=32),
            kernel_size=hp.Choice(f'conv_{i}_kernel', values=[3, 5]),
            activation='relu',
            padding='same'
        ))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(
        units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    optimizers = {
        'adam': Adam(),
        'rmsprop': RMSprop(),
        'sgd': SGD()
    }

    model.compile(
        optimizer=optimizers[optimizer],
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [22]:
tuner = kt.RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='result',
    project_name='actionclassifier'
)

tuner.search(train_generator, epochs=10, validation_data=validation_generator)

if len(tuner.oracle.get_best_trials()) == 0:
    raise ValueError("No trials were completed")

best_hps = tuner.get_best_hyperparameters()[0]

print(f"""Conv blocks: {best_hps.get('conv_blocks')},
Filters: {[best_hps.get(f'conv_{i}_filters') for i in range(best_hps.get('conv_blocks'))]},
Kernel sizes: {[best_hps.get(f'conv_{i}_kernel') for i in range(best_hps.get('conv_blocks'))]},
Dense units: {best_hps.get('dense_units')},
Dropout rate: {best_hps.get('dropout')},
Optimizer: {best_hps.get('optimizer')}.""")

best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(train_generator, epochs=10, validation_data=validation_generator)

best_model.save('model.h5')

files.download('model.h5')

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

Trial 10 Complete [00h 06m 09s]
val_accuracy: 0.8138801455497742

Best val_accuracy So Far: 0.8485804200172424
Total elapsed time: 00h 49m 00s
Conv blocks: 2,
Filters: [128, 64],
Kernel sizes: [5, 3],
Dense units: 128,
Dropout rate: 0.30000000000000004, 
Optimizer: rmsprop.
Epoch 1/10
80/80 [==============================] - 35s 418ms/step - loss: 0.4611 - accuracy: 0.7895 - val_loss: 0.5780 - val_accuracy: 0.6782
Epoch 2/10
80/80 [==============================] - 34s 421ms/step - loss: 0.3432 - accuracy: 0.8541 - val_loss: 0.4851 - val_accuracy: 0.7587
Epoch 3/10
80/80 [==============================] - 35s 435ms/step - loss: 0.2841 - accuracy: 0.8843 - val_loss: 0.9566 - val_accuracy: 0.6420
Epoch 4/10
80/80 [==============================] - 34s 421ms/step - loss: 0.2619 - accuracy: 0.8902 - val_loss: 0.5825 - val_accuracy: 0.7413
Epoch 5/10
80/80 [==============================] - 38s 473ms/step - loss: 0.2319 - accuracy: 0.9040 - val_loss: 0.5138 - val_accuracy: 0.7666
Epoch 6/10

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

25/25 [==============================] - 8s 310ms/step - loss: 0.3555 - accuracy: 0.8692
Test accuracy: 0.8692
Test loss: 0.3555


In [23]:
train_generator.class_indices

{'Run': 0, 'Walk': 1}

In [27]:
def process_image(image_path):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = best_model.predict(img_array)
    if prediction[0] > 0.5:
        print("Walking.")
    else:
        print("Running.")

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (150, 150))
        img_array = img_to_array(resized_frame)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

        prediction = best_model.predict(img_array)
        if prediction[0] > 0.5:
            class_label = 'Walking'
        else:
            class_label = 'Running'

        cv2.putText(frame, f'Prediction: {class_label}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

source = '/content/run.mp4'
file_extension = os.path.splitext(source)[1].lower()

if file_extension in ['.jpg', '.jpeg', '.png', '.bmp']:
    process_image(source)
elif file_extension in ['.mp4', '.avi', '.mov', '.mkv']:
    process_video(source)
else:
    print("Unsupported file type.")

1/1 [==============================] - 0s 20ms/step


KeyboardInterrupt: 